In [1]:
# import Libraries

import os
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document  # Correctly import Document class
from langchain_community.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
# Embedding Technique of OPENAI

import streamlit as st

embeddings= OpenAIEmbeddings(api_key = st.secrets['OPENAI_API_KEY'])

vectors = embeddings.embed_query('Welcome to Incredible INDIA.')

len(vectors)

C:\Users\rddvn\AppData\Local\Temp\ipykernel_23672\3639277972.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings= OpenAIEmbeddings(api_key = st.secrets['OPENAI_API_KEY'])


1536

In [3]:
from pinecone import Pinecone
import streamlit as st

pc = Pinecone(api_key = os.environ['PINE_CONE_KEY'])
index = pc.Index("vector-embeddings")
print(index.describe_index_stats())

# Initialize OpenAI LLM and Embeddings
embeddings = OpenAIEmbeddings(api_key = st.secrets['OPENAI_API_KEY'])

c:\Users\rddvn\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 169}},
 'total_vector_count': 169}


In [ ]:
## RAG process completely using OpenAI

import openai

# Set your OpenAI API key
openai.api_key = st.secrets['OPENAI_API_KEY']

def retrieve_query(query, k = 3):
    # Generate embeddings for the query
    query_embedding = embeddings.embed_query(query)
    
    # Query Pinecone for the top k similar documents
    results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    
    # Extract the documents from the results and wrap them in Document objects
    matching_results = [Document(page_content=result['metadata']['text']) for result in results['matches']]

    print(matching_results)
    return matching_results


def generate_text_gpt4(prompt, max_tokens = 400):
    # Use openai.chat.completions.create() instead of openai.Completion.create()
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Please avoid using the '*' symbol in your answer"},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        n=1,  # number of completions to generate
        stop=None,  # stop sequence (optional)
        temperature=0.7  # control creativity (0 to 1)
    )
    # Access the content of the generated text
    return response.choices[0].message.content

def answer_from_doc_search(doc_search, query):
    # Combine the documents from doc_search into a single prompt
    combined_docs = "\n\n".join(doc.page_content for doc in doc_search)  # Adjusted to doc.page_content to access text correctly
    prompt = f"Answer the question concisely and precisely based on the following documents. Please avoid using the '*' symbol in your answer.\n\nQuestion: {query}\n\nDocuments:\n{combined_docs}"

    # Generate the answer using GPT-4
    answer = generate_text_gpt4(prompt)
    return answer

# Example usage

query = "list all wifi devices"
doc_search = retrieve_query(query)

generated_answer = answer_from_doc_search(doc_search, query)
print(generated_answer)




[Document(metadata={}, page_content='**Product Category:** Mesh Wi-Fi Routers & Systems\n**Vendor:** Google Store\n**Promotion:** 25th-anniversary deals on Pixel devices (ending 9/23).\n**Products Mentioned:**\n* **Google Wifi:**\n    * Description: A smart buy for reliable Wi-Fi.\n    * Band Connectivity: Dual-band (2.4 and 5 GHz)\n    * Top Speeds: Up to 1.2 Gbps\n    * Performance: AC1200\n    * Wi-Fi Coverage: 1500 sq ft per router\n    * Number of Ethernet Ports: 2 per router\n    * Compatibility: Not compatible with Nest Wifi Pro.\n    * Materials: 49% post-consumer recycled content across its plastic \nmechanical parts. \n* **Nest Wifi Pro:**\n    * Description: Our fastest, most powerful system with Wi-Fi 6E.\n    * Band Connectivity: Tri-band (enhanced 2.4, 5, 6 GHz)\n    * Top Speeds: Up to 5.4 Gbps\n    * Performance: AXE5400\n    * Wi-Fi Coverage: 2200 sq ft per router\n    * Number of Ethernet Ports: 2 per router\n    * Compatibility: Not compatible with older generations 

In [4]:
## RAG process using llamma3 & OpenAI
 
from langchain_groq import ChatGroq


# Set the ChatGroq with environment protected API key and desired model
llm = ChatGroq(
    temperature = 0.7,
    groq_api_key = os.environ['GROQ_API_KEY'],  # Use environment variable for security
    model_name = "llama3-70b-8192"
)

system_instruction = "You are a helpful assistant."

def retrieve_query(query, k = 3):
    # Ensure embeddings and index are properly defined and replace with your embedding model and vector store details
    query_embedding = embeddings.embed_query(query)
    results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    
    matching_results = [result['metadata']['text'] for result in results['matches']]
    
    print(matching_results)
    return matching_results


def generate_text_groq(prompt, max_tokens = 400):
    response = llm.invoke(prompt)
    return response.content  # Access content directly


def answer_from_doc_search(doc_search, query):
    combined_docs = "\n\n".join(doc for doc in doc_search)
    prompt = f"{system_instruction}\n\nAnswer the question concisely and accurately based on the following documents.\n\nQuestion: {query}\n\nDocuments:\n{combined_docs} in structured and readable format "

    answer = generate_text_groq(prompt)
    return answer

query = "which all devices support C-type cable ?"
doc_search = retrieve_query(query)
generated_answer = answer_from_doc_search(doc_search, query)
print(generated_answer)

['* 1m USB-C to USB-C cable (USB 2.0)\n* Quick Start Guide\n* Quick Switch Adapter\n* SIM tool\n* (30W USB-C Power Charger sold separately)\nSpecial Offers (at the time of the webpage snapshot):\n* 3 months of YouTube Premium\n* 6 months of Google One with 2TB cloud storage\nCarrier Compatibility:\n* Unlocked, works with most major carriers\n* eSIM support\n* Carrier activation available with AT&T and Verizon through Google Store\nAdditional Information:\n* Preferred Care available for purchase\n* Trade-in program available\n* Google Store financing options\n* Free shipping (lowest cost option)\n', 'thermostats. \n* **Nest Power Connector:**  Alternative to C-wire for power connection.\n**Additional Information:**\n* Free shipping on orders over $35.\n* Financing options available.\n* Device recycling program.\n* Google is committed to making sustainable and accessible products.\n* Voice control requires a compatible smart device.\n* Nest Renew\'s "prioritize less expensive energy" is 

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = st.secrets['OPENAI_API_KEY']

def generate_text_gpt4(prompt, max_tokens=200):

    # Use openai.chat.completions.create() instead of openai.Completion.create()
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        n=1,  # number of completions to generate
        stop=None,  # stop sequence (optional)
        temperature=0.7  # control creativity (0 to 1)
    )
    # Access the content of the generated text
    return response.choices[0].message.content

# Example usage
prompt = "Write short 10 facts on India"
generated_text = generate_text_gpt4(prompt)
print(generated_text)